# AgentCore ランタイムでの大規模マルチモーダルペイロードの処理

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore ランタイムが Excel ファイルや画像などのマルチモーダルコンテンツを含む最大 100MB のペイロードをどのように処理するかを説明します。AgentCore ランタイムは、リッチメディアコンテンツや大規模データセットをシームレスに処理するように設計されています。

### チュートリアルの詳細

|情報| 詳細|
|:--------------------|:---------------------------------------------------------------------------------|
| チュートリアルの種類       | 大規模ペイロード & マルチモーダル処理|
| エージェントの種類          | 単一         |
| エージェントフレームワーク   | Strands Agents |
| LLM モデル           | Anthropic Claude Sonnet 4 |
| チュートリアルのコンポーネント | 大規模ファイル処理、画像分析、Excel データ処理 |
| チュートリアルの分野   | データ分析 & マルチモーダル AI                                                   |
| 例の複雑さ  | 中級                                                     |
| 使用 SDK            | Amazon BedrockAgentCore Python SDK|

### 主な機能

* **大規模ペイロードのサポート**: 最大 100MB のファイルを処理
* **マルチモーダル処理**: Excel ファイル、画像、テキストを同時に処理
* **データ分析**: 構造化データと視覚的コンテンツから洞察を抽出
* **Base64 エンコーディング**: JSON ペイロードを通じたバイナリデータの安全な送信

## 前提条件

* Python 3.10+
* AWS 認証情報が設定済み
* Docker が実行中
* テスト用のサンプル Excel ファイルと画像

In [ ]:
#!uv add -r requirements.txt --active
#!uv add openpyxl --active

import os
os.environ["AWS_PROFILE"] = "cline2"

## サンプルデータファイルの作成

大きなペイロード処理を実演するために、サンプルの Excel ファイルと画像ファイルを作成しましょう：

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import os

# サンプル販売データを含む大きな Excel ファイルを作成する
np.random.seed(42)
data = {
    'Date': pd.date_range('2023-01-01', periods=1000, freq='h'),
    'Product': np.random.choice(['Widget A', 'Widget B', 'Widget C', 'Gadget X', 'Gadget Y'], 1000),
    'Sales': np.random.randint(1, 1000, 1000),
    'Revenue': np.random.uniform(10.0, 5000.0, 1000),
    'Region': np.random.choice(['North', 'South', 'East', 'West'], 1000),
    'Customer_ID': np.random.randint(1000, 9999, 1000)
}

df = pd.DataFrame(data)
df.to_excel('large_sales_data.xlsx', index=False)

# サンプルチャート画像を作成する
img = Image.new('RGB', (600, 500), color='white')
draw = ImageDraw.Draw(img)

# 簡単な棒グラフを描画する
products = ['Widget A', 'Widget B', 'Widget C', 'Gadget X', 'Gadget Y']
values = [250, 180, 320, 150, 280]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']

max_value = max(values)
bar_width = 120
start_x = 100

for i, (product, value, color) in enumerate(zip(products, values, colors)):
    x = start_x + i * (bar_width + 20)
    height = int((value / max_value) * 400)
    y = 500 - height
    
    # Draw bar

# バーを描画する
    draw.rectangle([x, y, x + bar_width, 500], fill=color)
    
    # ラベルを追加する（フォントなしの簡略版）
    draw.text((x + 10, 510), product[:8], fill='black')
    draw.text((x + 10, y - 20), str(value), fill='black')

draw.text((300, 50), 'Sales Performance by Product', fill='black')
img.save('sales_chart.png')

# ファイルサイズの確認
excel_size = os.path.getsize('large_sales_data.xlsx') / (1024 * 1024)  # MB

# MB
image_size = os.path.getsize('sales_chart.png') / (1024 * 1024)  # MB

# MB

print(f"Excel file size: {excel_size:.2f} MB")
print(f"Image file size: {image_size:.2f} MB")
print(f"Total payload size: {excel_size + image_size:.2f} MB")

## マルチモーダルエージェントの作成

大きなペイロードから Excel ファイルと画像の両方を処理できるエージェントを作成しましょう：

In [ ]:
%%writefile multimodal_data_agent.py
from strands import Agent, tool
from strands.models import BedrockModel
import pandas as pd
import base64
import io
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

# モデルとエージェントを初期化する
#model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0"
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(
    model_id=model_id,
    max_tokens=16000
)

agent = Agent(
    model=model,
    system_prompt="""
    あなたは大きな Excel ファイルや画像を処理できるデータ分析アシスタントです。
    マルチモーダルデータが与えられた場合、構造化データと視覚的コンテンツの両方を分析し、
    両方のデータソースを組み合わせた包括的な洞察を提供します。
    """
)

@app.entrypoint
def multimodal_data_processor(payload, context):
    """
    Excel データと画像を含む大規模なマルチモーダルペイロードを処理します。
    
    Args:
        payload: プロンプト、excel_data（base64）、image_data（base64）を含みます
        context: ランタイムコンテキスト情報
    
    Returns:
        str: 両方のデータソースからの分析結果
    """
    prompt = payload.get("prompt", "Analyze the provided data.")
    excel_data = payload.get("excel_data", "")
    image_data = payload.get("image_data", "")
    
    print(f"=== Large Payload Processing ===")
    print(f"Session ID: {context.session_id}")
    
    if excel_data:
        print(f"Excel data size: {len(excel_data) / 1024 / 1024:.2f} MB")
    if image_data:
        print(f"Image data size: {len(image_data) / 1024 / 1024:.2f} MB")
    print(f"Excel data {excel_data}")
    print(f"Image data {image_data}")
    print(f"=== Processing Started ===")
    # base64 をバイトにデコードする
    excel_bytes = base64.b64decode(excel_data)
    # base64 をバイトにデコードする
    image_bytes = base64.b64decode(image_data)
    
    # データコンテキストを強化したプロンプト
    enhanced_prompt = f"""{prompt}
    両方のデータソースを分析して、洞察を提供してください。
    """
    
    response = agent(
        [{
            "document": {
                "format": "xlsx",
                "name": "excel_data",
                "source": {
                    "bytes": excel_bytes
                }
            }
        },
        {
            "image": {
                "format": "png",
                "source": {
                    "bytes": image_bytes
                }
            }
        },
        {
            "text": enhanced_prompt
        }]
    )
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

## インフラストラクチャのセットアップとエージェントのデプロイ

In [ ]:
import sys
import os

# 現在のノートブックのディレクトリを取得する
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

# utils.py の場所に移動する
utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.join(utils_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# sys.path に追加する
sys.path.insert(0, utils_dir)

from utils import create_agentcore_role

agent_name = "multimodal_data_agent"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="multimodal_data_agent.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)

launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

In [ ]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(f"Deployment status: {status}")

print(f"Final status: {status}")

## 大規模なマルチモーダルペイロードのテスト

ここでは、Excel データと画像の両方を含む大規模なペイロードでエージェントをテストしてみましょう：

In [ ]:
import base64
import uuid
import json
from IPython.display import Markdown, display

# ファイルを base64 にエンコードする
with open('large_sales_data.xlsx', 'rb') as f:
    excel_base64 = base64.b64encode(f.read()).decode('utf-8')

with open('sales_chart.png', 'rb') as f:
    image_base64 = base64.b64encode(f.read()).decode('utf-8')

# 大きなペイロードを作成する
large_payload = {
    "prompt": "Analyze the sales data from the Excel file and correlate it with the chart image. Provide insights on sales performance and trends.",
    "excel_data": excel_base64,
    "image_data": image_base64
}

session_id = str(uuid.uuid4())
print(f"📊 Processing large multi-modal payload...")
print(f"📋 Session ID: {session_id}")
print(f"📄 Excel size: {len(excel_base64) / 1024 / 1024:.2f} MB")
print(f"🖼️ Image size: {len(image_base64) / 1024 / 1024:.2f} MB")
print(f"📦 Total payload: {len(json.dumps(large_payload)) / 1024 / 1024:.2f} MB\n")

# 大きなペイロードでエージェントを呼び出す
invoke_response = agentcore_runtime.invoke(
    large_payload,
    session_id=session_id
)

# 修正部分: バイトを連結してからデコードする
response_bytes = b''
for r in invoke_response['response']:
    response_bytes += r

try:
    final_response = response_bytes.decode("utf-8")
    response_data = json.loads(final_response)
    display(Markdown(response_data))
except UnicodeDecodeError as e:
    print(f"デコードエラー: {e}")
    print("別の方法でデコードを試みます...")
    
    # エラー処理オプション1: エラーを無視して置換文字を使用
    final_response = response_bytes.decode("utf-8", errors="replace")
    try:
        response_data = json.loads(final_response)
        display(Markdown(response_data))
    except json.JSONDecodeError:
        print("JSONとして解析できません。レスポンスの形式を確認してください。")
        
        # エラー処理オプション2: レスポンスの形式を確認
        print(f"レスポンスの型: {type(invoke_response['response'][0])}")
        print(f"最初の数バイト: {invoke_response['response'][0][:20] if len(invoke_response['response']) > 0 else 'レスポンスが空です'}")
        
        # バイナリデータの可能性がある場合
        try:
            from PIL import Image
            import io
            img = Image.open(io.BytesIO(response_bytes))
            print("画像データとして処理します")
            display(img)
        except Exception as img_error:
            print(f"画像として処理できません: {img_error}")
            
            # 最後の手段: 生のバイトデータを保存
            with open(f"response_{session_id}.bin", "wb") as f:
                f.write(response_bytes)
            print(f"レスポンスをバイナリファイル 'response_{session_id}.bin' として保存しました")


## リソースのクリーンアップ

In [ ]:
import boto3

# AWS リソースのクリーンアップ
agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)
iam_client = boto3.client('iam')

# AgentCore ランタイムの削除
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

# ECR リポジトリの削除
ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# IAM ロールの削除
policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        PolicyName=policy_name
    )

iam_client.delete_role(RoleName=agentcore_iam_role['Role']['RoleName'])

# ローカルファイルのクリーンアップ
os.remove('large_sales_data.xlsx')
os.remove('sales_chart.png')

print("✅ Cleanup completed!")

# おめでとうございます！

Amazon Bedrock AgentCore Runtime で大規模なマルチモーダルペイロードの処理に成功しました！

## 学んだこと：

### 大規模ペイロード処理
* **100MB サポート**： AgentCore Runtime は最大 100MB のペイロードを処理できます
* **Base64 エンコーディング**： JSON ペイロードを通じたバイナリデータの安全な送信
* **効率的な処理**： 大規模データ処理に最適化されたランタイム

### マルチモーダル機能
* **Excel 分析**： スプレッドシートからの構造化データの処理
* **画像処理**： 視覚的なコンテンツやチャートの分析
* **複合分析**： 複数のデータタイプからの洞察の相関付け

### 主な利点
* **豊富なデータ処理**： 複雑なマルチフォーマットのデータセットを処理
* **スケーラブルなアーキテクチャ**： 大規模なワークロード向けに設計されたランタイム
* **ツール統合**： 特殊なデータ処理のためのカスタムツール
* **エンタープライズ対応**： 機密性の高いビジネスデータの安全な取り扱い

これは、AgentCore Runtime が複数のデータモダリティを持つエンタープライズ規模のデータ処理タスクを処理する能力を示しており、複雑なビジネスインテリジェンスやデータ分析アプリケーションに最適です。